# Text Translation and Sentiment Analysis using Transformers

## Project Overview:

The objective of this project is to analyze the sentiment of movie reviews in three different languages - English, French, and Spanish. We have been given 30 movies, 10 in each language, along with their reviews and synopses in separate CSV files named `movie_reviews_eng.csv`, `movie_reviews_fr.csv`, and `movie_reviews_sp.csv`.

- The first step of this project is to convert the French and Spanish reviews and synopses into English. This will allow us to analyze the sentiment of all reviews in the same language. We will be using pre-trained transformers from HuggingFace to achieve this task.

- Once the translations are complete, we will create a single dataframe that contains all the movies along with their reviews, synopses, and year of release in all three languages. This dataframe will be used to perform sentiment analysis on the reviews of each movie.

- Finally, we will use pretrained transformers from HuggingFace to analyze the sentiment of each review. The sentiment analysis results will be added to the dataframe. The final dataframe will have 30 rows


The output of the project will be a CSV file with a header row that includes column names such as **Title**, **Year**, **Synopsis**, **Review**, **Review Sentiment**, and **Original Language**. The **Original Language** column will indicate the language of the review and synopsis (*en/fr/sp*) before translation. The dataframe will consist of 30 rows, with each row corresponding to a movie.

In [1]:
# imports
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer
from transformers import pipeline

### Get data from `.csv` files and then preprocess data

In [38]:
# use the `pd.read_csv()` function to read the movie_review_*.csv files into 3 separate pandas dataframes

# Note: All the dataframes would have different column names. For testing purposes
# you should have the following column names/headers -> [Title, Year, Synopsis, Review]

def preprocess_data() -> pd.DataFrame:
    """
    Reads movie data from .csv files, map column names, add the "Original Language" column,
    and finally concatenate in one resultant dataframe called "df".
    """
    df_eng = pd.read_csv("C:/Temp/data/movie_reviews_eng.csv",skiprows=1,header=None)
    df_fr = pd.read_csv("C:/Temp/data/movie_reviews_fr.csv",skiprows=1, header=None)
    df_sp = pd.read_csv("C:/Temp/data/movie_reviews_sp.csv",skiprows=1, header=None)
    # add original language column

    df_eng["OriginalLanguage"] = "English"
    df_fr["OriginalLanguage"] = "French"
    df_sp["OriginalLanguage"] = "Spanish"

    # concatenate data frames

    df = pd.concat([df_eng, df_fr, df_sp])

    headers = ['Title', 'Year', 'Synopsis', 'Review', 'OriginalLanguage']
    df.columns = headers
    return df

df = preprocess_data()

In [39]:
df.head(20)

,Title,Year,Synopsis,Review,OriginalLanguage
0,The Shawshank Redemption,1994,"Andy Dufresne (Tim Robbins), a successful bank...","""The Shawshank Redemption is an inspiring tale...",English
1,The Dark Knight,2008,Batman (Christian Bale) teams up with District...,"""The Dark Knight is a thrilling and intense su...",English
2,Forrest Gump,1994,Forrest Gump (Tom Hanks) is a simple man with ...,"""Forrest Gump is a heartwarming and inspiratio...",English
3,The Godfather,1972,Don Vito Corleone (Marlon Brando) is the head ...,"""The Godfather is a classic movie that stands ...",English
4,Inception,2010,Dom Cobb (Leonardo DiCaprio) is a skilled thie...,"""Inception is a mind-bending and visually stun...",English
5,Blade Runner 2049,2017,"Officer K (Ryan Gosling), a new blade runner f...","""Boring and too long. Nothing like the origina...",English
6,Scott Pilgrim vs. the World,2010,Scott Pilgrim (Michael Cera) must defeat his n...,"""It was difficult to sit through the whole thi...",English
7,The Nice Guys,2016,"In 1970s Los Angeles, a private eye (Ryan Gosl...","""The Nice Guys tries too hard to be funny, and...",English
8,Solo: A Star Wars Story,2018,A young Han Solo (Alden Ehrenreich) joins a gr...,"""Dull and pointless, with none of the magic of...",English
9,The Island,2005,In a future where people are cloned for organ ...,"""The Island is a bland and forgettable sci-fi ...",English


In [14]:
df.sample(10)

,Title,Year,Synopsis,Review,Original Language
9,Babylon A.D.,2008,"Dans un futur lointain, un mercenaire doit esc...","""Ce film est un gâchis complet. Les personnage...",French
8,Les Visiteurs en Amérique,2000,Dans cette suite de la comédie française Les V...,"""Le film est une perte de temps totale. Les bl...",French
4,Le Fabuleux Destin d'Amélie Poulain,2001,Cette comédie romantique raconte l'histoire d'...,"""Le Fabuleux Destin d'Amélie Poulain est un fi...",French
7,The Nice Guys,2016,"In 1970s Los Angeles, a private eye (Ryan Gosl...","""The Nice Guys tries too hard to be funny, and...",English
6,La Tour Montparnasse Infernale,2001,Deux employés de bureau incompétents se retrou...,"""Je ne peux pas croire que j'ai perdu du temps...",French
0,The Shawshank Redemption,1994,"Andy Dufresne (Tim Robbins), a successful bank...","""The Shawshank Redemption is an inspiring tale...",English
1,La Casa de Papel,(2017-2021),Esta serie de televisión española sigue a un g...,"""La Casa de Papel es una serie emocionante y a...",Spanish
1,Intouchables,2011,Ce film raconte l'histoire de l'amitié improba...,"""Intouchables est un film incroyablement touch...",French
8,Solo: A Star Wars Story,2018,A young Han Solo (Alden Ehrenreich) joins a gr...,"""Dull and pointless, with none of the magic of...",English
3,El Laberinto del Fauno,2006,"Durante la posguerra española, Ofelia (Ivana B...","""El Laberinto del Fauno es una película fascin...",Spanish


### Text translation

Translate the **Review** and **Synopsis** column values to English.

In [15]:
pip install sentencepiece

Note: you may need to restart the kernel to use updated packages.


In [33]:
# load translation models and tokenizers
from transformers import MarianMTModel, MarianTokenizer

# TODO 2: Update the code below
fr_en_model_name = "Helsinki-NLP/opus-mt-fr-en"
es_en_model_name = "Helsinki-NLP/opus-mt-es-en"
fr_en_model = MarianMTModel.from_pretrained(fr_en_model_name)
es_en_model = MarianMTModel.from_pretrained(es_en_model_name)
fr_en_tokenizer = MarianTokenizer.from_pretrained(fr_en_model_name)
es_en_tokenizer = MarianTokenizer.from_pretrained(es_en_model_name)

# TODO 3: Complete the function below


def translate(text: str, model, tokenizer) -> str:
    """
    function to translate a text using a model and tokenizer
    """
    # encode the text using the tokenizer
    
    inputs = tokenizer(text, return_tensors="pt", padding=True)

    # generate the translation using the model
 
    outputs = model.generate(**inputs)

    # decode the generated output and return the translated text
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded

c:\Users\jblampain\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\models\marian\tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30 entries, 0 to 9
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Title             30 non-null     object
 1   Year              30 non-null     object
 2   Synopsis          30 non-null     object
 3   Review            30 non-null     object
 4   OriginalLanguage  30 non-null     object
dtypes: object(5)
memory usage: 1.4+ KB


In [42]:
# filter reviews in French and translate to English

print(f"Translated from French to English: {fr_reviews_en}")

Translated from French to English: 0    "The Land is an absolutely beautiful film with...
1    "Untouchables is an incredibly touching film w...
2    "Amélie is an absolutely charming film that wi...
3    "The Choristes are a beautiful film that will ...
4    "The Fabulous Destiny of Amélie Poulain is an ...
5    "I didn't like this movie at all. The concept ...
6    "I can't believe I've wasted time watching thi...
7    "This film is a complete disappointment. The j...
8    "The film is a total waste of time. The jokes ...
9    "This film is a complete mess. The characters ...
Name: Review, dtype: object


In [46]:
# TODO 4: Update the code below

import warnings
warnings.filterwarnings('ignore')

# filter reviews in French and translate to English
fr_reviews = df[df.OriginalLanguage == 'French']['Review']
fr_reviews_en = fr_reviews.apply(lambda x: translate(x, model=fr_en_model, tokenizer=fr_en_tokenizer))


# filter synopsis in French and translate to English
fr_synopsis = df[df.OriginalLanguage == 'French']['Synopsis']
fr_synopsis_en = fr_synopsis.apply(lambda x: translate(x, model=fr_en_model, tokenizer=fr_en_tokenizer))

# filter reviews in Spanish and translate to English
es_reviews = df[df.OriginalLanguage == 'Spanish']['Review']
es_reviews_en = es_reviews.apply(lambda x: translate(x, model=es_en_model, tokenizer=es_en_tokenizer))

# filter synopsis in Spanish and translate to English
es_synopsis = df[df.OriginalLanguage == 'Spanish']['Synopsis']
es_synopsis_en = es_synopsis.apply(lambda x: translate(x, model=es_en_model, tokenizer=es_en_tokenizer))


# update dataframe with translated text
# add the translated reviews and synopsis - you can overwrite the existing data
df.loc[df.OriginalLanguage == 'French', 'Review'] = fr_reviews_en
df.loc[df.OriginalLanguage == 'French', 'Synopsis'] = fr_synopsis_en
df.loc[df.OriginalLanguage == 'Spanish', 'Review'] = es_reviews_en
df.loc[df.OriginalLanguage == 'Spanish', 'Synopsis'] = es_synopsis_en




In [47]:
df.sample(10)
#print(f"Translated from French to English: {fr_reviews_en}")

,Title,Year,Synopsis,Review,OriginalLanguage
3,The Godfather,1972,Don Vito Corleone (Marlon Brando) is the head ...,"""The Godfather is a classic movie that stands ...",English
8,Les Visiteurs en Amérique,2000,In this continuation of the French comedy The ...,"""The film is a total wast of time. The jokes a...",French
9,El Incidente,2014,"In this Mexican land film, a group of people g...","""The Incident is a boring and frightless film ...",Spanish
6,Scott Pilgrim vs. the World,2010,Scott Pilgrim (Michael Cera) must defeat his n...,"""It was difficult to sit through the whole thi...",English
7,The Nice Guys,2016,"In 1970s Los Angeles, a private eye (Ryan Gosl...","""The Nice Guys tries too hard to be funny, and...",English
2,Y tu mamá también,2001,Two teenage friends (Gael García Bernal and Di...,"""And your mom is also a movie that stays with ...",Spanish
1,Intouchables,2011,This film tells the story of the unlikely frie...,"""Untouchables is an incredibly touching film w...",French
0,Roma,2018,Cleo (Yalitza Aparicio) is a young domestic em...,"""Rome is a beautiful and moving film that pays...",Spanish
9,The Island,2005,In a future where people are cloned for organ ...,"""The Island is a bland and forgettable sci-fi ...",English
0,La La Land,2016,This musical tells the story of a budding even...,"""The Land is an absolute beautiful film with s...",French


### Sentiment Analysis

Use HuggingFace pretrained model for sentiment analysis of the reviews. Store the sentiment result **Positive** or **Negative** in a new column titled **Sentiment** in the dataframe.

In [57]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.linear_model import LogisticRegression
import torch
import transformers

In [58]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [59]:
# TODO 5: Update the code below
# load sentiment analysis model
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
sentiment_classifier = AutoModelForSequenceClassification.from_pretrained(model_name)


# TODO 6: Complete the function below
def analyze_sentiment(text, classifier):
    """
    function to perform sentiment analysis on a text using a model
    Args:
        text (str): Input text for sentiment analysis.
        classifier: Pretrained sentiment analysis model.
    Returns:
        str: Sentiment label - Postive or Negative
    
    """
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)

    # Get model predictions
    with torch.no_grad():
        logits = classifier(**inputs).logits
        predicted_class = logits.argmax(dim=1).item()

    # Map predicted class to sentiment label
    sentiment_labels = ["negative","positive"]
    sentiment = sentiment_labels[predicted_class]

    return sentiment



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [60]:
# TODO 7: Add code below for sentiment analysis
# perform sentiment analysis on reviews and store results in new column

df['Sentiment'] = df['Review'].apply(lambda x: analyze_sentiment(x, sentiment_classifier))



In [61]:
df.sample(30)

,Title,Year,Synopsis,Review,OriginalLanguage,Sentiment
0,Roma,2018,Cleo (Yalitza Aparicio) is a young domestic em...,"""Rome is a beautiful and moving film that pays...",Spanish,negative
4,Inception,2010,Dom Cobb (Leonardo DiCaprio) is a skilled thie...,"""Inception is a mind-bending and visually stun...",English,negative
8,Les Visiteurs en Amérique,2000,In this continuation of the French comedy The ...,"""The film is a total wast of time. The jokes a...",French,negative
6,La Tour Montparnasse Infernale,2001,Two incompetent office workers found themselfs...,"""I can't believe I've been time watching this ...",French,negative
6,Scott Pilgrim vs. the World,2010,Scott Pilgrim (Michael Cera) must defeat his n...,"""It was difficult to sit through the whole thi...",English,negative
7,El Bar,2017,A group of people are trapped in a bar after M...,"""The Bar is a ridiculous and meaningless film ...",Spanish,negative
9,El Incidente,2014,"In this Mexican land film, a group of people g...","""The Incident is a boring and frightless film ...",Spanish,negative
0,La La Land,2016,This musical tells the story of a budding even...,"""The Land is an absolute beautiful film with s...",French,negative
2,Y tu mamá también,2001,Two teenage friends (Gael García Bernal and Di...,"""And your mom is also a movie that stays with ...",Spanish,negative
9,Babylon A.D.,2008,"In the distant future, a mercenary has to esco...","""This film is a complete mess. The characters ...",French,negative


In [56]:
# export the results to a .csv file
df.to_csv("c:/Temp/result/reviews_with_sentiment.csv", index=False)